In [82]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

In [102]:
# Importing and looking at the raw data
df = pd.read_csv('FIADB_API_Estimates_149290473.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153969 entries, 0 to 153968
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   COUNTY_CODE_AND_NAME  153969 non-null  object 
 1   SPECIES               153969 non-null  object 
 2   INVENTORY_YEAR        153969 non-null  object 
 3   ESTIMATE              153969 non-null  float64
 4   VARIANCE              153969 non-null  float64
 5   PLOT_COUNT            153969 non-null  int64  
 6   SE                    153969 non-null  float64
 7   SE_PERCENT            153969 non-null  float64
dtypes: float64(4), int64(1), object(3)
memory usage: 9.4+ MB


In [104]:
# Dropping the Total and State Subtotal rows
df = df[df['COUNTY_CODE_AND_NAME']!='Total']

# Removing all of the Total Species rows
df = df[df['SPECIES']!='Total']

# Removing all Total values from the Inventory Year column
df = df[df['INVENTORY_YEAR']!='Total']

# Changing inventory year from object to integer
df['INVENTORY_YEAR'] = df['INVENTORY_YEAR'].astype('int')


df.head()

,COUNTY_CODE_AND_NAME,SPECIES,INVENTORY_YEAR,ESTIMATE,VARIANCE,PLOT_COUNT,SE,SE_PERCENT
47688,01001 AL Autauga,eastern redcedar,2022,1.208065e+06,1.438944e+12,1,1.199560e+06,99.295942
47689,01001 AL Autauga,shortleaf pine,2022,1.429510e+06,2.096786e+12,1,1.448028e+06,101.295424
47690,01001 AL Autauga,slash pine,2022,9.089185e+05,8.145408e+11,1,9.025192e+05,99.295942
47691,01001 AL Autauga,loblolly pine,2022,2.286141e+07,2.417957e+14,4,1.554978e+07,68.017593
47692,01001 AL Autauga,red maple,2022,2.980979e+06,4.672755e+12,3,2.161656e+06,72.514958


In [105]:
# Splitting the State abbreviations from the 
states = [i.split(' ')[1] for i in df['COUNTY_CODE_AND_NAME']]
states

# Splitting off the rest of the county state code and joining multiple word county names with a space
counties = [' '.join(i.split(' ')[2:]) for i in df['COUNTY_CODE_AND_NAME']]
counties

# Setting state and county columns to the states and counties lists
df['STATE'] = states
df['COUNTY'] = counties

# Changing cases from mostly all lowercase to titlecase for a cleaner report
df['SPECIES'] = [i.title() for i in df['SPECIES']]

# Dropping County Code and Name column
df = df.drop(['COUNTY_CODE_AND_NAME'], axis = 1).reset_index(drop = True)

df

,SPECIES,INVENTORY_YEAR,ESTIMATE,VARIANCE,PLOT_COUNT,SE,SE_PERCENT,STATE,COUNTY
0,Eastern Redcedar,2022,1.208065e+06,1.438944e+12,1,1.199560e+06,99.295942,AL,Autauga
1,Shortleaf Pine,2022,1.429510e+06,2.096786e+12,1,1.448028e+06,101.295424,AL,Autauga
2,Slash Pine,2022,9.089185e+05,8.145408e+11,1,9.025192e+05,99.295942,AL,Autauga
3,Loblolly Pine,2022,2.286141e+07,2.417957e+14,4,1.554978e+07,68.017593,AL,Autauga
4,Red Maple,2022,2.980979e+06,4.672755e+12,3,2.161656e+06,72.514958,AL,Autauga
...,...,...,...,...,...,...,...,...,...
106276,Mockernut Hickory,2012,4.449438e+05,2.065216e+11,1,4.544465e+05,102.135703,TN,Polk
106277,Pignut Hickory,2012,1.636718e+05,2.794491e+10,1,1.671673e+05,102.135703,TN,Polk
106278,Sourwood,2012,2.502343e+04,6.532038e+08,1,2.555785e+04,102.135703,TN,Polk
106279,White Oak,2012,1.476349e+06,2.273701e+12,1,1.507880e+06,102.135703,TN,Polk


In [106]:
# Only keeping the columns I need for the Tableau report
output_df = df[['SPECIES', 'ESTIMATE', 'INVENTORY_YEAR','STATE', 'COUNTY']]
output_df

,SPECIES,ESTIMATE,INVENTORY_YEAR,STATE,COUNTY
0,Eastern Redcedar,1.208065e+06,2022,AL,Autauga
1,Shortleaf Pine,1.429510e+06,2022,AL,Autauga
2,Slash Pine,9.089185e+05,2022,AL,Autauga
3,Loblolly Pine,2.286141e+07,2022,AL,Autauga
4,Red Maple,2.980979e+06,2022,AL,Autauga
...,...,...,...,...,...
106276,Mockernut Hickory,4.449438e+05,2012,TN,Polk
106277,Pignut Hickory,1.636718e+05,2012,TN,Polk
106278,Sourwood,2.502343e+04,2012,TN,Polk
106279,White Oak,1.476349e+06,2012,TN,Polk


In [113]:
# Max year for each of the states
year_max = output_df[['INVENTORY_YEAR', 'STATE']].groupby(['STATE']).max().reset_index()
year_max

,STATE,INVENTORY_YEAR
0,AL,2022
1,AR,2021
2,GA,2021
3,IL,2021
4,IN,2021
5,KY,2019
6,MO,2021
7,MS,2021
8,NC,2021
9,SC,2021


In [115]:
# Data from the latest year ONLY
latest_df = output_df.merge(year_max, left_on = 'STATE', right_on = 'STATE')
latest_df

,SPECIES,ESTIMATE,INVENTORY_YEAR_x,STATE,COUNTY,INVENTORY_YEAR_y
0,Eastern Redcedar,1.208065e+06,2022,AL,Autauga,2022
1,Shortleaf Pine,1.429510e+06,2022,AL,Autauga,2022
2,Slash Pine,9.089185e+05,2022,AL,Autauga,2022
3,Loblolly Pine,2.286141e+07,2022,AL,Autauga,2022
4,Red Maple,2.980979e+06,2022,AL,Autauga,2022
...,...,...,...,...,...,...
106276,Mockernut Hickory,4.449438e+05,2012,TN,Polk,2019
106277,Pignut Hickory,1.636718e+05,2012,TN,Polk,2019
106278,Sourwood,2.502343e+04,2012,TN,Polk,2019
106279,White Oak,1.476349e+06,2012,TN,Polk,2019


In [116]:
latest_df.to_csv('tn_surrounding_species.csv')